In [ ]:
import time
import edist.ted as ted
import numpy as np
import hyperelastic_laws as hyperelastic
import torch

In [ ]:
task = 'hyperelastic_discovery'

T = 3125*6
N = 32
N_test = 100

learning_rate = 1E-3
weight_decay = 1e-4
print_step = 100

In [ ]:
import recursive_tree_grammar_auto_encoder as rtg_ae
dim = 128
dim_vae  = 8
num_params = 0
model = rtg_ae.TreeGrammarAutoEncoder(hyperelastic.grammar, dim = dim, dim_vae = dim_vae)
for key in model.state_dict():
    num_params += model.state_dict()[key].numel()
print('The model contains %d parameters' % num_params)

In [ ]:
training_data = []
for j in range(N * T):
    nodes, adj = hyperelastic.sample_tree()
    training_data.append((nodes, adj))
test_data = []
for j in range(N_test):
    nodes, adj = hyperelastic.sample_tree()
    test_data.append((nodes, adj))
start_time = time.time()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
for epoch in range(1, T + 1):
    optimizer.zero_grad()
    minibatch_loss = 0.
    for j in range(N*(epoch-1), N*epoch):
        nodes, adj = training_data[j]
        loss = model.compute_loss(nodes, adj, beta = 0.0001, sigma_scaling = 0.0001)
        minibatch_loss += loss.item()
        loss.backward()
    optimizer.step()
    minibatch_loss /= N
    avg_loss = minibatch_loss
    if(epoch % print_step == 0):
        print('loss avg after %d data points: %g' % (epoch * N, avg_loss))
print('finished training after %g seconds' % (time.time() - start_time))
sqe = 0.
for nodes, adj in test_data:
    _, z = model.encode(nodes, adj)
    nodes_act, adj_act, _ = model.decode(z, max_size = 100)
    d = ted.ted(nodes, adj, nodes_act, adj_act)
    sqe += d * d
rmse = np.sqrt(sqe / N_test)
print('obtained RMSE of %g' % rmse)
# torch.save(model.state_dict(), 'results/%s_%s.torch' % (task, model))

In [ ]:
sqe = 0.
for nodes, adj in test_data:
    # encode it
    _, z = model.encode(nodes, adj)
    nodes_act, adj_act, _ = model.decode(z, max_size = 100)
    d = ted.ted(nodes, adj, nodes_act, adj_act)
    sqe += d * d
    print(hyperelastic.to_algebraic_string(nodes,adj))
    print(hyperelastic.to_algebraic_string(nodes_act,adj_act))
rmse = np.sqrt(sqe / N_test)
print('obtained RMSE of %g' % rmse)